# ChromaのPythonライブラリをインストール

In [ ]:
!pip install chromadb

In [ ]:
!pip install -U transformers

# ライブラリとデータセットをロード

In [ ]:
import pandas as pd
import chromadb

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.info()  # Chromaにデータを入れる際、ChromaのIDはstrかつユニークのみ受け付ける

In [ ]:
df.head()

今回はA１のタブレットをカスタムEmbeddingを使い探せるようにする


# Chromaの使い方(Custom Embedding Function)

## Step 1: ChromaのデータをDiskにセーブ

In [ ]:
# testのフォルダーが作られ、chroma.sqlite3が作成される
chroma_client = chromadb.PersistentClient(path="test")  # Pathはセーブしたいロケーション　（今回はGoogle Colabなので名前だけ)

## Step 2:　カスタムEmbeddingを作成する
今回はLLAMAを使用  
*SentenceTransfotmerを推奨  
APIを使えばOpenAI等のEmneddingを使えるが、従来これらのモデルはエンコード用に作られていない。

HuggingFace: https://huggingface.co/tasks/sentence-similarity  
Chroma: https://docs.trychroma.com/docs/embeddings/embedding-functions

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings  # 上書きする為に必要
from sentence_transformers import SentenceTransformer # HuggingFaceのライブラリーを使う為

#　上書きするのは__call__
class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents, model_name: str = 'thuan9889/llama_embedding_model_v1') -> Embeddings:
        sentences = input

        model = SentenceTransformer(model_name)  # モデルのロード
        embeddings = model.encode(sentences)  # インプットをEmbedding

        # リストにコンバート
        embeddings_as_list = [embedding.tolist() for embedding in embeddings]

        return embeddings_as_list

## Step 3: Vector　DatabaseをカスタムEmbeddingと一緒に作成

In [ ]:
myembedding = CustomEmbeddingFunction()  #　インスタンス化
collection = chroma_client.get_or_create_collection(
    name="test",
    embedding_function=myembedding) #　ディフォルトのembedding_functionを変更

## Step 4: データをCollectionに加える

In [ ]:
collection.add(
    documents=df['Item'].tolist(),
    ids=df['ID'].tolist()
)

In [ ]:
collection.peek()

## Step 4: Queryを書く

In [ ]:
results = collection.query(
    query_texts=["Tablet", "Printer"], # 探したいキーワードを入れる
    n_results=3 # 欲しい数を入れる：　ディフォルトは10個
)
results